In [146]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import sys
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import cross_val_score, KFold

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g

Предварительный анализ данных производится в файле data_exploration.ipynb

Загрузим исходный датасет

In [147]:
df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")

In [148]:
df.columns

Index(['repository_name', 'commit_hash', 'commit_date', 'commit_author',
       'commit_message', 'bugs'],
      dtype='object')

Удалим столбец, не несущий никакой полезной инфы

In [149]:
df.drop(columns=["commit_hash"], inplace=True)

Преобразуем категориальные признаки в числовые

In [150]:
# OHE для имени репозитория

df = pd.concat([df, pd.get_dummies(df.repository_name)], axis=1)
df.drop(columns=["repository_name"], inplace=True)

In [151]:
# закодируем столбец commit_author, в данном случае я сделаю обычный OHE

df = f_g.ohe(df, "commit_author")  # функция выполняет действия аналогичные ячейке выше

In [152]:
# данные на текущий момент
df.sample(3)

,commit_date,commit_message,bugs,agent,conductor,dockers,mlm,sensor,standard,Alice,Bob,Carol,Dabe,Eve,Mallory,Peggy,Trudy,Victor,Wendy
179,2020-03-20T10:24:33,return simple OK,2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
174,2020-03-20T17:05:56,Diff remote,2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
135,2020-04-13T12:57:17,compose configs moved to traps/docker,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Сгенерируем признаки из столбца отвечающего за время коммита

In [153]:
# так как изначально тип данного столбца - object
df["commit_date"] = pd.to_datetime(df.commit_date)

df = f_g.encode_work_days(df)
df = f_g.encode_work_hours(df)

In [154]:
df.sample(3)

,commit_date,commit_message,bugs,agent,conductor,dockers,mlm,sensor,standard,Alice,...,Eve,Mallory,Peggy,Trudy,Victor,Wendy,no_work_d,work_d,no_work_h,work_h
150,2020-04-06 16:36:00,Popen service enabling/disabling; removing ima...,6,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
75,2020-05-22 16:08:34,Обращение к главному серверу,3,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
227,2020-05-12 16:53:07,Авторизация в ЕЛК,2,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,1


In [155]:
if "commit_date" in df.columns:
    df.drop(columns=["commit_date"], inplace=True)

Сначала получим бэйзлан без какой-либо информации из сообщения коммита

In [156]:
df_without_msg = df.drop(columns=["commit_message"])

In [157]:
X = df_without_msg.drop(columns=["bugs"])
y = df_without_msg.bugs

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_idxs, test_idxs = X_train.index, X_test.index

Отмасштабируем значения

In [159]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

Посмотрим качество на разных базовых моделях

In [160]:
zoo_models = [
    LinearRegression(),
    SGDRegressor(random_state=42),
    Ridge(random_state=42),
    Lasso(random_state=42),
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR()
    ]

for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()
    

Модель - LinearRegression()
Среднее квадратное отклонение: 3.599230424388977

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 2.792461669579143

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 2.813940971574099

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 2.6581221678556193

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 2.7976837028211925

Модель - SVR()
Среднее квадратное отклонение: 2.547624178147599



In [162]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
splits = cv.split(X)

eval_data_lst = []
for i_train, i_test in splits:
    

    X_train, X_test, y_train, y_test = X.loc[i_train], X.loc[i_test], y[i_train], y[i_test]

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)

    eval_data_lst.append((X_train, X_test, y_train, y_test))

for model in zoo_models:
    print(f"Модель - {model}")

    tmp_lst = []

    for X_train, X_test, y_train, y_test in eval_data_lst:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        tmp_lst.append(mean_squared_error(y_test, preds))
    print(f"Среднее квадратное отклонение: {np.mean(tmp_lst)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print() 

Модель - LinearRegression()
Среднее квадратное отклонение: 3.061867391378663

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 2.8427464535834384

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 2.8261025740434538

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 3.4774931163795175

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 2.8155012197320235

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 2.8420382249740155

Модель - SVR()
Среднее квадратное отклонение: 3.0814115021828874



Теперь посмотрим, как отработают модели, если мы попробуем извлечь информацию из сообщения комита самым простым способом с помощью tfidf и svd/pca

In [163]:
tfidf = TfidfVectorizer(tokenizer=d_u.simple_tokenizer)

msg_embs_train = tfidf.fit_transform(df.commit_message.values[train_idxs]).toarray()
msg_embs_test = tfidf.transform(df.commit_message.values[test_idxs]).toarray()
print(msg_embs_train.shape)
print(msg_embs_test.shape)

(255, 555)
(64, 555)


In [164]:
X = df.drop(columns=["bugs", "commit_message"])
y = df.bugs

msg_embs_train = pd.DataFrame(msg_embs_train)
msg_embs_test = pd.DataFrame(msg_embs_test)
# X = pd.concat([X, msg_embs], axis=1)

X_train, X_test, y_train, y_test = X.loc[train_idxs], X.loc[test_idxs], y[train_idxs], y[test_idxs]

msg_embs_train.index = train_idxs
msg_embs_test.index = test_idxs

X_train_emb = pd.concat([X_train, msg_embs_train], axis=1)
X_test_emb = pd.concat([X_test, msg_embs_test], axis=1)

In [165]:
for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train_emb.to_numpy(), y_train.to_numpy())
    preds = model.predict(X_test_emb.to_numpy())
    print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднее квадратное отклонение: 4.466638540535732

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 2.3360978058935036

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 2.5210254767340077

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 1.9212890625

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 1.9642904281494462

Модель - SVR()
Среднее квадратное отклонение: 2.060367996474572



Можем заметить, что при добавлении информации из сообщения коммита с помощью tfidf мы получили наименьшее отклонение от целевой переменной с помощью моделей RandomForestRegressor и GradientBoostingRegressor. Теперь посмотрим на средние ошибки на кросс-валидации

In [177]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
splits = cv.split(X)

In [178]:
eval_data_lst = []
for i_train, i_test in splits:
    tfidf = TfidfVectorizer(tokenizer=d_u.simple_tokenizer)

    msg_embs_train = tfidf.fit_transform(df.commit_message.values[i_train]).toarray()
    msg_embs_test = tfidf.transform(df.commit_message.values[i_test]).toarray()

    msg_embs_train = pd.DataFrame(msg_embs_train)
    msg_embs_test = pd.DataFrame(msg_embs_test)
    # X = pd.concat([X, msg_embs], axis=1)

    X_train, X_test, y_train, y_test = X.loc[i_train], X.loc[i_test], y[i_train], y[i_test]

    msg_embs_train.index = i_train
    msg_embs_test.index = i_test

    X_train_emb = pd.concat([X_train, msg_embs_train], axis=1)
    X_test_emb = pd.concat([X_test, msg_embs_test], axis=1)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_emb)
    X_test = ss.transform(X_test_emb)

    eval_data_lst.append((X_train_emb, X_test_emb, y_train, y_test))    

c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. 

In [180]:
for model in zoo_models:
    print(f"Модель - {model}")

    tmp_lst = []

    for X_train_emb, X_test_emb, y_train, y_test in eval_data_lst:
        model.fit(X_train_emb.to_numpy(), y_train)
        preds = model.predict(X_test_emb.to_numpy())
        tmp_lst.append(mean_squared_error(y_test, preds))
    print(f"Среднее квадратное отклонение: {np.mean(tmp_lst)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднее квадратное отклонение: 4.051258697962091

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 2.544314136103807

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 2.569979525885581

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 3.4774931163795175

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 2.55626306547619

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 2.3125281732052922

Модель - SVR()
Среднее квадратное отклонение: 2.75252931290019



In [95]:
dim_red_comps[0]

(sklearn.decomposition._truncated_svd.TruncatedSVD, 5)

Модель - LinearRegression()
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________5
Среднее квадратное отклонение: 2.707965850830078
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________10
Среднее квадратное отклонение: 2.6955108642578125
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________20
Среднее квадратное отклонение: 2.68939208984375
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________50
Среднее квадратное отклонение: 2.6620330810546875
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________100
Среднее квадратное отклонение: 2.7361907958984375
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________300
Среднее квадратное отклонение: 2.6362152099609375
<class 'sklearn.decomposition._pca.PCA'>________5
Среднее квадратное отклонение: 2.624267578125
<class 'sklearn.decomposition._pca.PCA'>________10
Среднее квадратное отклонение: 2.6888427734375
<class 'sklearn.decomposition._pca.PCA'>________20
Среднее к